In [ ]:
!pip install duckdb-engine

In [4]:
import os

#git_author_email = os.environ["GIT_AUTHOR_EMAIL"]
#git_author_name = os.environ["GIT_AUTHOR_NAME"]
#git_committer_email = os.environ["GIT_COMMITTER_EMAIL"]
#git_committer_name = os.environ["GIT_COMMITTER_NAME"]
#git_password = os.environ["GIT_PASSWORD"]
#git_username = os.environ["GIT_USERNAME"]


In [5]:
import requests
import zipfile
import xml.etree.ElementTree as ET
import pandas as pd
import matplotlib.pyplot as plt
import json
import duckdb
import sqlalchemy
import time
from io import BytesIO

In [24]:
sslverify=True #nastavit na False u serveru, ktere hlasi chybu ssl

urlRoot = "https://volby.cz/opendata/"
urlRootDavky = "https://volby.cz/pls/"
opendatafileHome = "opendata.htm"
davky="vysledky_okrsky?davka="

## Prezident 2018

In [ ]:
volby = "prez2018"
ciselniky = "PREZ2018ciselniky20171215_csv.zip"
registry = "PREZ2018reg20180127_csv.zip"
okrskovaData = "PREZ2018data20180127_csv_kolo2.zip"
urlOpendata = urlRoot+volby+"/"+volby+"_"+opendatafileHome
urlDavky = urlRootDavky+volby+"/"+davky
urlCiselniky = urlRoot+volby+"/"+ciselniky
urlRegistry = urlRoot+volby+"/"+registry
urlOkrskovaData = urlRoot+volby+"/"+okrskovaData
davky=True
kola=True

In [25]:
if not os.path.exists(volby):
    os.mkdir(volby)

In [26]:
def get_posledni_davka(urlDavky):
    posledni_davka = requests.get(url_davky,verify=sslverify)
    root = ET.fromstring(posledni_davka.text)
    #for child in root:
    #    print (child.tag,child.attrib)
    davka = root[0].attrib
    davky_pocet = int(davka.get("PORADI_DAVKY")) + 1
    return(davky_pocet)

In [27]:
davky_pocet_kolo1 = get_posledni_davka(url_davky+"&kolo=1")
davky_pocet_kolo2 = get_posledni_davka(url_davky+"&kolo=2")
print(davky_pocet_kolo1,davky_pocet_kolo2)

54 39


In [ ]:
def get_data_voleb (volby,davky,kola,ciselniky,registry,okrskovaData)

In [28]:
def get_davky(kolo,davky_pocet):
    davkydir = os.path.join(volby,"davky"+kolo)
    dir_exists = os.path.exists(davkydir)
    if not dir_exists:
        os.mkdir(davkydir)
    for i in range(1,davky_pocet):
        url=url_davky+str(i)+"&kolo="+kolo
        response=requests.get(url,verify=sslverify)
        time.sleep(1)
        print("davka",i,response.status_code,"\r")
        if i<10:
            davkaurl=os.path.join(davkydir,"davka_0"+str(i)+".xml")                                
        else:
            davkaurl=os.path.join(davkydir,"davka_"+str(i)+".xml")
        with open(davkaurl,"wb") as file:
            file.write(response.content)
get_davky("1",davky_pocet_kolo1)
get_davky("2",davky_pocet_kolo2)          

In [ ]:
def get_cisregokr():
    for url in [url_ciselniky,url_registry,url_okrskova_data]:
        response = requests.get(url,verify=sslverify)
        zipfile.ZipFile(BytesIO(response.content)).extractall(path=volby)
get_cisregokr()

In [29]:
response = requests.get(url_ciselniky)


In [ ]:
response.url.split("/")[-1]

In [ ]:
def zpracuj_davku(davkafile): #zpracování xml souboru dávky, vrací dict dávky, volebních dat okrsků, výsledků hlasování okrsků
    davka_list = []
    davky_list = []
    okrsek_list = []
    okrsky_list = []
    hlasy_list = []

    it = ET.iterparse(davkafile)
    for _, el in it:
        el.tag = el.tag.split('}', 1)[1]  # strip all namespaces
    root = it.root
    cas = list(root.items()[1])
    davka_list.append(cas)
    for element in root:
        if element.tag == "DAVKA":
            for davka_attr in element.items():
                davka_list.append(list(davka_attr))
            davky_list.append(dict(davka_list))
        elif element.tag == "OKRSEK":
            okrsek_dict = {}
            for okrsek_attr in element.items():
                okrsek_list.append(list(okrsek_attr))
                okrsek_dict.update(dict(okrsek_list))
            for okrsek_data in element:
                #print(okrsek_data.keys())
                if(okrsek_data.tag) == "UCAST_OKRSEK":
                    okrsek_dict.update(okrsek_data.attrib)
                else:
                    cis_obec=okrsek_dict["CIS_OBEC"]
                    cis_okrsek=okrsek_dict["CIS_OKRSEK"]
                    okrsek_hlasy=[]
                    for strana in okrsek_data.items():
                       okrsek_hlasy.append(strana[1])
                    hlasy_list.append([cis_obec,cis_okrsek,okrsek_hlasy[0],okrsek_hlasy[1]])
            okrsky_list.append(okrsek_dict)
    davka_df = pd.DataFrame(davky_list,index=["PORADI_DAVKY"])
    davka_okrsky_df = pd.DataFrame(okrsky_list)
    davka_hlasy_df = pd.DataFrame(hlasy_list,dtype='uint32',columns=["CIS_OBEC","CIS_OKRSEK","STRANA","HLASY"])

    return(davka_df,davka_okrsky_df,davka_hlasy_df)


In [ ]:
 
davky_dir = "davky"
davky = os.listdir(davky_dir)
davky.sort()
i=1
for davka_file in davky:
    if davka_file.endswith("xml"):
        xmldavka = os.path.join(davky_dir,davka_file)
        print(davka_file
              ,end='\r'
              )
        davka_df,okrsek_df,hlasy_davka_df = zpracuj_davku(xmldavka)
        if i == 1:
            davky_df = davka_df.astype({"DATUM_CAS_GENEROVANI":"datetime64"})
            #.astype()
            davky_okrsky_df = okrsek_df.astype({"DATUM_CAS_ZPRAC":"datetime64"})
            davky_hlasy_df = hlasy_davka_df
        else:
            davky_df = pd.concat([davky_df,davka_df]).astype({"DATUM_CAS_GENEROVANI":"datetime64"
            ,"PORADI_DAVKY":"uint8","OKRSKY_DAVKA":"uint16","OKRSKY_CELKEM":"uint16","OKRSKY_ZPRAC":"uint16"})
            davky_okrsky_df = pd.concat([davky_okrsky_df,okrsek_df]).astype({"DATUM_CAS_ZPRAC":"datetime64","CIS_OBEC":"uint32"
            ,"CIS_OKRSEK":"uint32","PORADI_ZPRAC":"uint16","OPAKOVANE":"bool","ZAPSANI_VOLICI":"uint32"
            ,"VYDANE_OBALKY":"uint32","ODEVZDANE_OBALKY":"uint32","PLATNE_HLASY":"uint32"})
            davky_hlasy_df = pd.concat([davky_hlasy_df,hlasy_davka_df]).astype("uint32")
            
        i=i+1
    


In [ ]:


# connect to an in-memory database
volby_davky = duckdb.connect()

# create the tables from the DataFrames
volby_davky.execute("CREATE TABLE davky AS SELECT * FROM davky_df")
volby_davky.execute("CREATE TABLE davky_okrsky AS SELECT * FROM davky_okrsky_df")
volby_davky.execute("CREATE TABLE davky_hlasy AS SELECT * FROM davky_hlasy_df")

# insert into the table "my_table" from the DataFrame "my_df"
#con.execute("INSERT INTO my_table SELECT * FROM my_df")

volby_davky.execute("DESCRIBE TABLE davky_hlasy").fetchall()

volby_davky.execute("SHOW TABLES").fetchall()

CSV_DIR = "csv_od"
csv_json_files = os.listdir(CSV_DIR)
csv_json_files.sort()
for file in csv_json_files:
    data,filetype = file.split(".")
    if file.endswith(".json"):
        df_name = data + "_df"
        with open(os.path.join(CSV_DIR,file), "r") as json_file:
            #df_name = pd.read_json(json_file)
            print("soubor:",file)
            json_data = json.load(json_file)
            for json_items in json_data.items():
                print(json_items)
            print()

            #s())
            #print(json_data.values())

In [ ]:
#test
cns_json = json.load(open('csv_od/cns.json'))
description = cns_json["dc:description"]
tableschema=cns_json["tableSchema"]
print(tableschema)
#print(tableschema["columns"])

cns_json_df = pd.DataFrame(tableschema["columns"])
cns_json_df